# FLIP (04): Learning Theory (I)
**(Module 03: Operation Research)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, but NOT allowed to change or distribute this package.

Prepared by and for 
**Student Members** |
2006-2018 [TULIP Lab](http://www.tulip.org.au), Australia

---

# Optimal Mixture and Blending Problems

This [IPython notebook](http://ipython.org/notebook.html) demonstrates the development of models to solve optimal blending and mixture problems using GLPK/MathProg.

### Initializations

In [ ]:
from IPython.core.display import HTML
HTML(open("styles/custom.css", "r").read())

## Background

Mixture and blending problems are among the most commonly encountered optimization problems in real world applications. The basic problem is to determine a blend (mixture) of components maximizing subject to constraints on the blend. Examples include diet, product blending, financial portfolios, and many, many others.

This notebook uses a the diet problem from [Chapter 2 of the AMPL book](http://www.ampl.com/BOOK/download.html) to demonstrate the formulation and solution of blending problems in MathProg. The problem is solved several times in order to demonstrate features of MathProg useful in constructing models that can be reused for other data sets, and extended to much larger applications.

## Diet Problem

We'll begin by reviewing data for  The data is organized as three [pandas DataFrames](http://pandas.pydata.org/) (some information on DataFrames is [here](http://pandas.pydata.org/pandas-docs/dev/dsintro.html) and [here](http://pandas.pydata.org/pandas-docs/dev/10min.html?highlight=panels)). For larger scale problems, this data  would be extracted from spreadsheets or databases.

### Weekly Nutrition Requirements

The first data set contains lower and upper bounds on weekly nutrition expressed a percentage of the recommended minimum daily requirements.

In [ ]:
import pandas as pd

req = {
    'A' : [700, 10000],
    'C' : [700, 10000],
    'B1': [700, 10000],
    'B2': [700, 10000]}

display(pd.DataFrame(req.values(), index = req.keys(), columns = ['n_min', 'n_max']))

### Nutrition Data

The second data set includes 'label data' from some typical package goods.

In [ ]:
nutr = {
    'BEF' : {'A': 60, 'C': 20, 'B1': 10, 'B2': 15},
    'CHK' : {'A':  8, 'C':  0, 'B1': 20, 'B2': 20},
    'FSH' : {'A':  8, 'C': 10, 'B1': 15, 'B2': 10},
    'HAM' : {'A': 40, 'C': 40, 'B1': 35, 'B2': 10},
    'MCH' : {'A': 15, 'C': 35, 'B1': 15, 'B2': 15},
    'MTL' : {'A': 70, 'C': 30, 'B1': 15, 'B2': 15},
    'SPG' : {'A': 25, 'C': 50, 'B1': 25, 'B2': 15},
    'TUR' : {'A': 60, 'C': 20, 'B1': 15, 'B2': 10}}

display(pd.DataFrame.from_dict(nutr).transpose())

### Price Data

The third data set provides price information for the packaged foods.

In [ ]:
price ={'BEF' : ['Beef', 3.19],
        'CHK' : ['Chicken', 2.59],
        'FSH' : ['Fish', 2.29],
        'HAM' : ['Ham', 2.89],
        'MCH' : ['Macaroni & Cheese', 1.89],
        'MTL' : ['Meat Loaf', 1.99],
        'SPG' : ['Spaghetti', 1.99],
        'TUR' : ['Turkey', 2.49]}

display(pd.DataFrame(price.values(), index = price.keys(), columns = ['food','price']))

### Problem Statement

Determine a shopping list that meets the nutrition requirements for minimum cost.

## A Basic Model

This first attempt at a model for the diet problem will encode the basic elements of the problem. We'll include a decision variable for each type of packaged good, and explicitly write out the objective function, and each of the nutrition requirements.

The following cell uses the `script` cell magic to run the MathProg model with the `glpsol` command line. This requires that you [install the `glpk` package](http://en.wikibooks.org/wiki/GLPK/Obtaining_GLPK) on the machine used to execute this notebook.

In [ ]:
%%script glpsol -m /dev/stdin -o /dev/stdout

var xBEF >= 0; 
var xCHK >= 0; 
var xFSH >= 0;
var xHAM >= 0; 
var xMCH >= 0; 
var xMTL >= 0;
var xSPG >= 0; 
var xTUR >= 0;

minimize Total_Cost:
    3.19*xBEF + 2.59*xCHK + 2.29*xFSH + 2.89*xHAM 
  + 1.89*xMCH + 1.99*xMTL + 1.99*xSPG + 2.49*xTUR;
    
subject to A:
    700 <= 60*xBEF +  8*xCHK +  8*xFSH + 40*xHAM 
         + 15*xMCH + 70*xMTL + 25*xSPG + 60*xTUR <= 10000;
    
subject to C:
    700 <= 20*xBEF +  0*xCHK + 10*xFSH + 40*xHAM
         + 35*xMCH + 30*xMTL + 50*xSPG + 20*xTUR <= 10000;
    
subject to B1:
    700 <= 10*xBEF + 20*xCHK + 15*xFSH + 35*xHAM
         + 15*xMCH + 15*xMTL + 25*xSPG + 15*xTUR <= 10000;
    
subject to B2:
    700 <= 15*xBEF + 20*xCHK + 10*xFSH + 10*xHAM
         + 15*xMCH + 15*xMTL + 15*xSPG + 10*xTUR <= 10000;

end;

The previous cell used a 'cell magic' to run the model, with the results placed into a file `output.txt`. This next cell uses standard python commands to read and display the output file.

### Questions

* What is the total cost of   What are we eating, and how much does it cost?
* Is this a realistic solution? How would like to modify the problem?
* What do the shadow costs (also called marginal costs, or sensitivities) mean?
* Can we apply this same model to other data sets?

## Extending the Model

### Introducing MathProg Sets

In this second version we introduce a set to represent all types of packaged foods. The use of a set allows us to declare all of the decision variables in one line. The decision variables now have subscripts associated with members from the set of foods.

In [ ]:
%%script glpsol -m /dev/stdin -o /dev/stdout --out output

set FOODS := {'BEF','CHK','FSH','HAM','MCH','MTL','SPG','TUR'};

var x{FOODS} >= 0;

minimize Total_Cost:
    3.19*x['BEF'] + 2.59*x['CHK'] + 2.29*x['FSH'] + 2.89*x['HAM'] 
  + 1.89*x['MCH'] + 1.99*x['MTL'] + 1.99*x['SPG'] + 2.49*x['TUR'];
    
subject to A:
    700 <= 60*x['BEF'] +  8*x['CHK'] +  8*x['FSH'] + 40*x['HAM']
         + 15*x['MCH'] + 70*x['MTL'] + 25*x['SPG'] + 60*x['TUR'] <= 10000;
    
subject to C:
    700 <= 20*x['BEF'] +  0*x['CHK'] + 10*x['FSH'] + 40*x['HAM']
         + 35*x['MCH'] + 30*x['MTL'] + 50*x['SPG'] + 20*x['TUR'] <= 10000;
    
subject to B1:
    700 <= 10*x['BEF'] + 20*x['CHK'] + 15*x['FSH'] + 35*x['HAM']
         + 15*x['MCH'] + 15*x['MTL'] + 25*x['SPG'] + 15*x['TUR'] <= 10000;
    
subject to B2:
    700 <= 15*x['BEF'] + 20*x['CHK'] + 10*x['FSH'] + 10*x['HAM']
         + 15*x['MCH'] + 15*x['MTL'] + 15*x['SPG'] + 10*x['TUR'] <= 10000;

end;

### Adding a Data Section

Next we introduce a set for the nutrients. Much of the problem data can be indexed by memebers of the set of nutrients, and this data is defined in the data section. 

In [ ]:
%%script glpsol -m /dev/stdin -o /dev/stdout --out output

set FOODS;
set NUTRS;

param price{FOODS} >= 0;
param n_min{NUTRS} >= 0;
param n_max{NUTRS} >= 0;

var x{FOODS} >= 0;

minimize Total_Cost: sum {f in FOODS} price[f]*x[f];
    
subject to A:
    n_min['A'] <= 60*x['BEF'] +  8*x['CHK'] +  8*x['FSH'] + 40*x['HAM']
                + 15*x['MCH'] + 70*x['MTL'] + 25*x['SPG'] + 60*x['TUR'] <= n_max['A'];
    
subject to C:
    n_min['C'] <= 20*x['BEF'] +  0*x['CHK'] + 10*x['FSH'] + 40*x['HAM']
                + 35*x['MCH'] + 30*x['MTL'] + 50*x['SPG'] + 20*x['TUR'] <= n_max['C'];
    
subject to B1:
    n_min['B1'] <= 10*x['BEF'] + 20*x['CHK'] + 15*x['FSH'] + 35*x['HAM']
                 + 15*x['MCH'] + 15*x['MTL'] + 25*x['SPG'] + 15*x['TUR'] <= n_max['B1'];
    
subject to B2:
    n_min['B2'] <= 15*x['BEF'] + 20*x['CHK'] + 10*x['FSH'] + 10*x['HAM']
                 + 15*x['MCH'] + 15*x['MTL'] + 15*x['SPG'] + 10*x['TUR'] <= n_max['B2'];
        
data;

param : FOODS : price :=
    BEF  3.19
    CHK  2.59
    FSH  2.29
    HAM  2.89
    MCH  1.89
    MTL  1.99
    SPG  1.99
    TUR  2.49;

param : NUTRS : n_min n_max :=
    A    700   10000
    C    700   10000
    B1   700   10000
    B2   700   10000;
    
end;

### Factoring the Model and Data

The factoring of the problem into separate model and data sections is completed by introducing a two dimensional data set for the nutrient content of the packaged foods. The parameter data is placed in the data section, and the model constraints written in terms of the indexed parameters.

In [ ]:
%%script glpsol -m /dev/stdin -o /dev/stdout --out output

set FOODS;
set NUTRS;

param price{FOODS} >= 0;
param n_min{NUTRS} >= 0;
param n_max{NUTRS} >= 0;
param a{FOODS,NUTRS} >= 0;

var x{FOODS} >= 0;

minimize Total_Cost: sum {f in FOODS} price[f]*x[f];

subject to n_req {n in NUTRS}:
    n_min[n] <= sum {f in FOODS} a[f,n] * x[f] <= n_max[n];

data;

param : FOODS : price :=
    BEF  3.19
    CHK  2.59
    FSH  2.29
    HAM  2.89
    MCH  1.89
    MTL  1.99
    SPG  1.99
    TUR  2.49;

param : NUTRS : n_min n_max :=
    A    700   10000
    C    700   10000
    B1   700   10000
    B2   700   10000;

param a :  A   C  B1  B2 :=
    BEF   60  20  10  15
    CHK    8   0  20  20
    FSH    8  10  15  10
    HAM   40  40  35  10
    MCH   15  35  15  15
    MTL   70  30  15  15
    SPG   25  50  25  15
    TUR   60  20  15  10;
    
end;

### Post-Processing Results

The output from glpk/MathProg can be enchanced by adding `printf` statements for selected data, or `table` statements for access to structured data indexed by sets.

In [ ]:
%%script glpsol -m /dev/stdin -o output.txt -y display.txt --out output

set FOODS;
set NUTRS;

param price{FOODS} >= 0;
param x_min{FOODS} >= 0;
param x_max{FOODS} >= 0;

param n_min{NUTRS} >= 0;
param n_max{NUTRS} >= 0;

param a{FOODS,NUTRS} >= 0;

var x{f in FOODS} >= 0;

minimize Total_Cost: sum {f in FOODS} price[f]*x[f];

subject to n_req {n in NUTRS}:
    n_min[n] <= sum {f in FOODS} a[f,n] * x[f] <= n_max[n];

solve;

printf "\nMinimum Cost Diet = %6.2f $/week.\n\n", Total_Cost;

table results {f in FOODS} OUT "CSV" "output.csv" "Table":
    f ~ Food,
    x[f] ~ Quantity,
    price[f] ~ Price;
    
data;

param : FOODS : price :=
    BEF  3.19
    CHK  2.59
    FSH  2.29
    HAM  2.89
    MCH  1.89
    MTL  1.99
    SPG  1.99
    TUR  2.49;

param : NUTRS : n_min n_max :=
    A    700   10000
    C    700   10000
    B1   700   10000
    B2   700   10000;

param a :  A   C  B1  B2 :=
    BEF   60  20  10  15
    CHK    8   0  20  20
    FSH    8  10  15  10
    HAM   40  40  35  10
    MCH   15  35  15  15
    MTL   70  30  15  15
    SPG   25  50  25  15
    TUR   60  20  15  10;
    
end;

In [ ]:
print output

In [ ]:
print(open('output.txt').read())

In [ ]:
print(open('display.txt').read())

In [ ]:
import pandas
df = pandas.read_csv('output.csv')
display(df)
df['Quantity'].plot(kind='bar').set_xticklabels(df['Food']);

## Adding Problem Features

The new model adds a number of additional features, including

* Force diversification by imposing bounds on the minimum and maximum number of packages.
* Constraining the solution to integer values

In [ ]:
%%script glpsol -m /dev/stdin -o display.txt --out output

set FOODS;
set NUTRS;

param price{FOODS} >= 0;
param x_min{FOODS} >= 0;
param x_max{FOODS} >= 0;

param n_min{NUTRS} >= 0;
param n_max{NUTRS} >= 0;

param a{FOODS,NUTRS} >= 0;

var x{f in FOODS} integer >= x_min[f], <= x_max[f];

minimize Total_Cost: sum {f in FOODS} price[f]*x[f];

subject to n_req {n in NUTRS}:
    n_min[n] <= sum {f in FOODS} a[f,n] * x[f] <= n_max[n];
    
solve;

printf "\nMinimum Cost Diet = %6.2f $/week.\n\n", Total_Cost;

table results {f in FOODS} OUT "CSV" "output.csv" "Table":
    f ~ Food,
    x[f] ~ Quantity,
    price[f] ~ Price;

data;

param : FOODS : price x_min x_max :=
    BEF  3.19  2  30
    CHK  2.59  2  30
    FSH  2.29  2  30
    HAM  2.89  2  30
    MCH  1.89  2  30
    MTL  1.99  2  30
    SPG  1.99  2  30
    TUR  2.49  2  30;

param : NUTRS : n_min n_max :=
    A     700   10000 
    C     700   10000
    B1    700   10000
    B2    700   10000
    NA      0   40000
    CAL 16000   24000;

param a :  A   C  B1  B2    NA  CAL :=
    BEF   60  20  10  15   938  295
    CHK    8   0  20  20  2180  770
    FSH    8  10  15  10   945  440
    HAM   40  40  35  10   278  430
    MCH   15  35  15  15  1182  315
    MTL   70  30  15  15   896  400
    SPG   25  50  25  15  1329  370
    TUR   60  20  15  10  1397  450;
    
end;

In [ ]:
print output

In [ ]:
print(open('display.txt').read())

In [ ]:
import pandas
df = pandas.read_csv('output.csv')
display(df)
df['Quantity'].plot(kind='bar').set_xticklabels(df['Food']);

## Exercises

1. Lowering the upper limit on the number of packages that can be purchased will eventually lead to an infeasible solution. Deliberately modify the constraints so that you see this behavior, and learn how to identify these situations.

2. Alter the table output to include the marginal values of the nutrient constraints. What nutrients have the highest marginal values?